In [ ]:
# NLTK Stands for Natural Language Toolkit
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize

# Import OS Library
import os

# Import string library
import string

# Import pandas
import pandas as pd
# Supress warnings
pd.options.mode.chained_assignment = None 

# Import numpy
import numpy as np

# Import math
import math

# Import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import (
    CountVectorizer
)

import warnings
warnings.filterwarnings('ignore')

C:\Users\ivopb\AppData\Roaming\Python\Python37\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
C:\Users\ivopb\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more d

## 1 - Preparing Features

### 1.1 - Loading the Data

**First we will load 1000 positive reviews and 1000 negative reviews:**

In [ ]:
neg_files = os.listdir('data/neg')[0:1000]
pos_files = os.listdir('data/pos')[0:1000]

In [ ]:
def get_reviews(directory, file_list):
    files=[]
    for file in file_list:
        f = open('data/'+directory+'/'+file, "r")
        #append each line in the file to a list
        files.append(f.read())
        f.close()
        
    return files

In [ ]:
negative_files = get_reviews('neg', neg_files)
positive_files = get_reviews('pos', pos_files)

Let's see some examples of some negative reviews:

In [ ]:
print(negative_files[1])

the happy bastard's quick movie review 
damn that y2k bug . 
it's got a head start in this movie starring jamie lee curtis and another baldwin brother ( william this time ) in a story regarding a crew of a tugboat that comes across a deserted russian tech ship that has a strangeness to it when they kick the power back on . 
little do they know the power within . . . 
going for the gore and bringing on a few action sequences here and there , virus still feels very empty , like a movie going for all flash and no substance . 
we don't know why the crew was really out in the middle of nowhere , we don't know the origin of what took over the ship ( just that a big pink flashy thing hit the mir ) , and , of course , we don't know why donald sutherland is stumbling around drunkenly throughout . 
here , it's just " hey , let's chase these people around with some robots " . 
the acting is below average , even from the likes of curtis . 
you're more likely to get a kick out of her work in hallow

In [ ]:
print(negative_files[2])

it is movies like these that make a jaded movie viewer thankful for the invention of the timex indiglo watch . 
based on the late 1960's television show by the same name , the mod squad tells the tale of three reformed criminals under the employ of the police to go undercover . 
however , things go wrong as evidence gets stolen and they are immediately under suspicion . 
of course , the ads make it seem like so much more . 
quick cuts , cool music , claire dane's nice hair and cute outfits , car chases , stuff blowing up , and the like . 
sounds like a cool movie , does it not ? 
after the first fifteen minutes , it quickly becomes apparent that it is not . 
the mod squad is certainly a slick looking production , complete with nice hair and costumes , but that simply isn't enough . 
the film is best described as a cross between an hour-long cop show and a music video , both stretched out into the span of an hour and a half . 
and with it comes every single clich ? . 
it doesn't really 

And some positive ones:

In [ ]:
print(positive_files[2])

you've got mail works alot better than it deserves to . 
in order to make the film a success , all they had to do was cast two extremely popular and attractive stars , have them share the screen for about two hours and then collect the profits . 
no real acting was involved and there is not an original or inventive bone in it's body ( it's basically a complete re-shoot of the shop around the corner , only adding a few modern twists ) . 
essentially , it goes against and defies all concepts of good contemporary filmmaking . 
it's overly sentimental and at times terribly mushy , not to mention very manipulative . 
but oh , how enjoyable that manipulation is . 
but there must be something other than the casting and manipulation that makes the movie work as well as it does , because i absolutely hated the previous ryan/hanks teaming , sleepless in seattle . 
it couldn't have been the directing , because both films were helmed by the same woman . 
i haven't quite yet figured out what i like

In [ ]:
print(positive_files[1])

every now and then a movie comes along from a suspect studio , with every indication that it will be a stinker , and to everybody's surprise ( perhaps even the studio ) the film becomes a critical darling . 
mtv films' _election , a high school comedy starring matthew broderick and reese witherspoon , is a current example . 
did anybody know this film existed a week before it opened ? 
the plot is deceptively simple . 
george washington carver high school is having student elections . 
tracy flick ( reese witherspoon ) is an over-achiever with her hand raised at nearly every question , way , way , high . 
mr . " m " ( matthew broderick ) , sick of the megalomaniac student , encourages paul , a popular-but-slow jock to run . 
and paul's nihilistic sister jumps in the race as well , for personal reasons . 
the dark side of such sleeper success is that , because expectations were so low going in , the fact that this was quality stuff made the reviews even more enthusiastic than they have 

### 1.2 - Pre-process Text

it's relevant to pre-process our text with some of the techniques we've learned so far. Here's our pipeline:
    - Lower case text.
    - Remove punctuation
    - Tokenize our reviews.
    - Remove stop words.
    - Stem our words.

In [ ]:
positive_files =[file.lower() for file in positive_files]
negative_files =[file.lower() for file in negative_files]

Let's build our function that removes the punctuation from our text:

In [ ]:
def remove_punct(list_of_text):
    
    no_punct = []
    
    for index, file in enumerate(list_of_text):
        file = (
            file.translate(
                str.maketrans('', '', string.punctuation)
            )
        )

        no_punct.append(file)
    return no_punct

In [ ]:
positive_files = remove_punct(positive_files)
negative_files = remove_punct(negative_files)

Removing our stop words:

In [ ]:
stop_words = stopwords.words('english')

In [ ]:
stop_words[1:10]

['me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [ ]:
positive_files_tokenized = [word_tokenize(file) for file in positive_files]
negative_files_tokenized = [word_tokenize(file) for file in negative_files]

In [ ]:
def remove_stop_words(tokenized_text):
    # Retrieve Stop Words
    stop_words = stopwords.words('english')
    
    list_files_no_stop = []
    
    for file in tokenized_text:
        no_stop_words = []
        for word in file:
            if word not in stop_words:
                no_stop_words.append(word)
        
        list_files_no_stop.append(
            no_stop_words
        )
        
    return list_files_no_stop

In [ ]:
positive_files_tokenized = remove_stop_words(positive_files_tokenized)
negative_files_tokenized = remove_stop_words(negative_files_tokenized)

Before stemming, let's do some analytics on our positive and negative reviews.

In [ ]:
# Let's flat our list into words:
flat_list_positive = [
    item for sublist in positive_files_tokenized for item in sublist
]

In [ ]:
print('We have {} words in our positive files - {} are distinct'
      .format(len(flat_list_positive), len(set(flat_list_positive))))

We have 374641 words in our positive files - 34781 are distinct


In [ ]:
# Let's flat our list into words:
flat_list_negative = [
    item for sublist in negative_files_tokenized for item in sublist
]

In [ ]:
print('We have {} words in our positive files - {} are distinct'
      .format(len(flat_list_negative), len(set(flat_list_negative))))

We have 335151 words in our positive files - 32595 are distinct


### 1.3 - Log Ratio of Words

We can rely on Freq Dist to check the most relevant words for positive and negative reviews:

In [ ]:
nltk.FreqDist(flat_list_negative).most_common(20)

[('film', 3994),
 ('movie', 3048),
 ('one', 2617),
 ('like', 1835),
 ('even', 1381),
 ('good', 1127),
 ('time', 1112),
 ('would', 1049),
 ('get', 1037),
 ('bad', 1019),
 ('much', 997),
 ('characters', 909),
 ('films', 903),
 ('story', 903),
 ('character', 893),
 ('plot', 876),
 ('two', 865),
 ('make', 818),
 ('first', 805),
 ('really', 787)]

In [ ]:
nltk.FreqDist(flat_list_positive).most_common(20)

[('film', 4855),
 ('one', 2903),
 ('movie', 2392),
 ('like', 1716),
 ('story', 1212),
 ('films', 1198),
 ('also', 1197),
 ('good', 1191),
 ('even', 1174),
 ('time', 1171),
 ('characters', 1038),
 ('much', 1024),
 ('character', 1010),
 ('would', 993),
 ('life', 983),
 ('well', 967),
 ('first', 963),
 ('two', 960),
 ('see', 952),
 ('way', 913)]

In [ ]:
negative_freq = pd.DataFrame(
    nltk.FreqDist(flat_list_negative).values(), 
    index=nltk.FreqDist(flat_list_negative).keys(),
    columns=['word_count_negative']
)
positive_freq = pd.DataFrame(
    nltk.FreqDist(flat_list_positive).values(), 
    index=nltk.FreqDist(flat_list_positive).keys(),
    columns=['word_count_positive']
)

In [ ]:
word_sentiment = negative_freq.join(positive_freq).fillna(0)

Let's calculate the Ratio between Positive and Negative Words

In [ ]:
word_sentiment['ratio'] = (
    np.log(
        word_sentiment.word_count_positive
        /word_sentiment.word_count_negative
    )
)

We can now check which words are more relevant for positive and negative reviews:

In [ ]:
word_sentiment.loc[
    word_sentiment.word_count_negative + word_sentiment.word_count_positive > 100 
].sort_values(by='ratio')

,word_count_negative,word_count_positive,ratio
godzilla,116,14.0,-2.114533
wasted,101,16.0,-1.842532
lame,90,15.0,-1.791759
waste,121,22.0,-1.704748
worst,259,48.0,-1.685627
...,...,...,...
jackie,47,209.0,1.492187
cameron,22,106.0,1.572397
toy,17,87.0,1.632695
titanic,18,105.0,1.763589


In [ ]:
positive_files_tokenized[0]

['films',
 'adapted',
 'comic',
 'books',
 'plenty',
 'success',
 'whether',
 'theyre',
 'superheroes',
 'batman',
 'superman',
 'spawn',
 'geared',
 'toward',
 'kids',
 'casper',
 'arthouse',
 'crowd',
 'ghost',
 'world',
 'theres',
 'never',
 'really',
 'comic',
 'book',
 'like',
 'hell',
 'starters',
 'created',
 'alan',
 'moore',
 'eddie',
 'campbell',
 'brought',
 'medium',
 'whole',
 'new',
 'level',
 'mid',
 '80s',
 '12part',
 'series',
 'called',
 'watchmen',
 'say',
 'moore',
 'campbell',
 'thoroughly',
 'researched',
 'subject',
 'jack',
 'ripper',
 'would',
 'like',
 'saying',
 'michael',
 'jackson',
 'starting',
 'look',
 'little',
 'odd',
 'book',
 'graphic',
 'novel',
 '500',
 'pages',
 'long',
 'includes',
 'nearly',
 '30',
 'consist',
 'nothing',
 'footnotes',
 'words',
 'dont',
 'dismiss',
 'film',
 'source',
 'get',
 'past',
 'whole',
 'comic',
 'book',
 'thing',
 'might',
 'find',
 'another',
 'stumbling',
 'block',
 'hells',
 'directors',
 'albert',
 'allen',
 'hugh

Finally we can Stem our words:

In [ ]:
def stem_sentence(tokenized_sentence):
    # Retrieve Stop Words
    stemmer = PorterStemmer()
    stem_sentence = []
    for word in tokenized_sentence:
        stem_sentence.append(stemmer.stem(word))
        
    return stem_sentence

In [ ]:
stemmed_negative_files = [
    stem_sentence(sentence) for sentence in negative_files_tokenized
]
stemmed_positive_files = [
    stem_sentence(sentence) for sentence in positive_files_tokenized
]

Join our sentences into a single text:

In [ ]:
sentence_negative = [' '.join(file) for file in stemmed_negative_files]
sentence_positive = [' '.join(file) for file in stemmed_positive_files]

In [ ]:
sentence_negative = pd.DataFrame(sentence_negative, columns=['sentence'])
sentence_positive = pd.DataFrame(sentence_positive, columns=['sentence'])

In [ ]:
sentence_positive.head(10)

,sentence
0,film adapt comic book plenti success whether t...
1,everi movi come along suspect studio everi ind...
2,youv got mail work alot better deserv order ma...
3,jaw rare film grab attent show singl imag scre...
4,moviemak lot like gener manag nfl team postsal...
5,june 30 1960 selftaught idealist yet pragmat y...
6,appar director toni kay major battl new line r...
7,one colleagu surpris told will see betsi wed s...
8,bloodi clash independ lumumba refus pander bel...
9,american action film slowli drown death sea as...


In [ ]:
training_base = sentence_positive.append(sentence_negative).reset_index(drop=True)

In [ ]:
cv = CountVectorizer(binary=True)
X = cv.fit_transform(training_base.sentence).todense()
X = pd.DataFrame(
    X,
    columns=cv.get_feature_names()
)

In [ ]:
X.shape

(2000, 31903)

Our y consists of labels with zeros and one's - 1 for positive reviews and 0 for negative reviews. 

In [ ]:
y = np.append(np.ones(1000), np.zeros(1000))

In [ ]:
X.columns

Index(['00', '000', '0009f', '000acr', '000aweek', '000foot', '000paltri',
       '007', '007esqu', '010',
       ...
       'zuehlk', 'zuko', 'zukovski', 'zulu', 'zundel', 'zurg', 'zweibel',
       'zwick', 'zwigoff', 'zyci'],
      dtype='object', length=31903)

### 2.2 Training Process

In [ ]:
X_test = X[0:100].append(X[-100:])
y_test = np.append(y[0:100],y[-100:])

And now selecting our training set, which will contain 1800 reviews:

In [ ]:
X_train = X[100:len(X)-100]
y_train = y[100:len(y)-100]

Note that with our huge dimensional space we have a high probability of overfitting our algorithm. Let's train our model:

In [ ]:
lm = LogisticRegression(random_state=1234)

In [ ]:
lm.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=1234, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

Let's check the accuracy on our train_set - we can use the predict method to predict using our trained model:

In [ ]:
y_pred = lm.predict(X_train)

In [ ]:
((y_pred == y_train).sum())/len(y_train)

1.0

Our accuracy is 100% on the training set. This is a result that will probably show us that we are overfitting our hypothesis - meaning that the function that we predicted (the weights) adjust perfectly to our training set. We are able to predict with 100% confidence the fact that a review is positive or negative.
<br>
<br>
To really evaluate the performance of our algorithm, we have to check the accuracy in the test set - the data that we left out of this training process and that mimics the real world that your algorithm should find.

In [ ]:
y_pred_test = lm.predict(X_test)

In [ ]:
(y_pred_test == y_test).sum()/len(y_test)

0.87

As you can see, our model behaves generally well but in the test set we achieve 87% accuracy - still a lot better than our first model with two features but with a difference of 0.13 percentage points in accuracy between train and test sets. 
<br>
<br>
Having this difference between training and test datasets is not optimum as it may lead to a quicker model degradation (i.e., when you have your model in production, the accuracy may get worse over time really quickly). To reduce the overfit problem, we could use a technique called regularization:

In [ ]:
lm_reg = LogisticRegression(penalty='l2', C=0.001)
lm_reg.fit(X_train, y_train)

LogisticRegression(C=0.001, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
y_pred = lm_reg.predict(X_train)
((y_pred == y_train).sum())/len(y_train)

0.9016666666666666

In [ ]:
y_pred_test = lm_reg.predict(X_test)
(y_pred_test == y_test).sum()/len(y_test)

0.86

Notice that, although our test set accuracy is lower with regularization the difference between target and test set are a much lower than in the solution without regularization. This will make your model a bit more stable when you want to productize it and use it with "real" new data.

Other ways to reduce overfit is to:
    - Give less training data to the model (not optimum);
    - Give less features to the model;
    - Try another type of model;